In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Environment Setup
# -----------------------------
np.random.seed(42)
torch.manual_seed(42)

# -----------------------------
# Preprocessing
# -----------------------------
# Generate random data (e.g., 1000 samples, 20 features)
X = np.random.rand(1000, 20)

# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------------
# Train-Test Split
# -----------------------------
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [3]:
# Train Base Model (Sparse Autoencoders)
# -----------------------------
class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim, l1_weight=1e-4):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, input_dim)
        self.l1_weight = l1_weight

    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return decoded

    def l1_penalty(self):
        return self.l1_weight * torch.norm(self.encoder.weight, 1)

input_dim = X_train.shape[1]
encoding_dim = 10  # Dimension of the bottleneck layer
model = SparseAutoencoder(input_dim, encoding_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        mse_loss = criterion(outputs, batch_X)
        l1_loss = model.l1_penalty()
        loss = mse_loss + l1_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [5/20], Loss: 1.0961
Epoch [10/20], Loss: 0.9820
Epoch [15/20], Loss: 0.9290
Epoch [20/20], Loss: 0.8967


In [4]:
# Planning (Simulated Experience Using Reconstructed Samples)
# -----------------------------
model.eval()
with torch.no_grad():
    reconstructed_samples = model(X_test_tensor)

# Example: Compare first 5 original and reconstructed samples
for i in range(5):
    print(f"Original:      {np.round(X_test[i], 3)}")
    print(f"Reconstructed: {np.round(reconstructed_samples[i].numpy(), 3)}\n")

# -----------------------------
# Fine-Tune Model
# -----------------------------
# Adjust learning rate and increase epochs for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=0.0005)
epochs_finetune = 10
for epoch in range(epochs_finetune):
    model.train()
    running_loss = 0.0
    for batch_X, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        mse_loss = criterion(outputs, batch_X)
        l1_loss = model.l1_penalty()
        loss = mse_loss + l1_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        print(f"[Fine-Tuning] Epoch [{epoch+1}/{epochs_finetune}], Loss: {running_loss/len(train_loader):.4f}")

# -----------------------------
# Evaluate
# -----------------------------
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    mse = criterion(test_outputs, X_test_tensor).item()
print(f"Test MSE: {mse:.4f}")

# -----------------------------
# Deploy Policy
# -----------------------------
# Save the trained model
model_path = "sparse_autoencoder_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Original:      [-1.072 -0.137  0.313 -0.626  0.991  1.146 -0.596  1.185 -1.453  1.208
 -1.144 -1.416  0.505 -0.095  1.412 -0.516 -0.706 -0.034 -0.145 -0.521]
Reconstructed: [0.088 0.059 0.053 0.194 0.469 0.126 0.12  0.547 0.065 0.161 0.057 0.069
 0.166 0.403 0.374 0.044 0.057 0.123 0.154 0.203]

Original:      [ 1.375  0.363  1.022 -0.273 -1.345 -0.266 -0.134  0.201 -1.446  1.738
 -0.829 -1.027  0.115  0.137 -0.232  0.91   0.103  0.344  0.78  -1.626]
Reconstructed: [0.475 0.351 0.199 0.096 0.162 0.287 0.141 0.396 0.069 0.264 0.172 0.113
 0.072 0.075 0.136 0.12  0.234 0.378 0.184 0.092]

Original:      [ 1.634  1.103  1.488 -1.286 -1.493  0.165 -0.711  1.149  0.554  1.667
 -0.793 -0.061  0.028 -0.921  0.296 -0.236 -0.144 -0.512 -1.662  0.093]
Reconstructed: [0.386 0.554 0.231 0.083 0.043 0.225 0.092 0.488 0.113 0.379 0.394 0.055
 0.104 0.047 0.215 0.116 0.403 0.173 0.077 0.095]

Original:      [-1.707  0.073  0.155  1.129  0.024  1.409  0.456  1.191 -1.295  1.268
  0.652 -1.122 -1.028  